# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f92f82dfeb0>
├── 'a' --> tensor([[ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939]])
└── 'x' --> <FastTreeValue 0x7f92f82dfdc0>
    └── 'c' --> tensor([[-1.8362, -2.1066, -0.7294, -1.0983],
                        [ 0.6755,  0.3028, -0.6359,  0.0524],
                        [-1.0733,  0.6447,  0.1886, -2.0146]])

In [4]:
t.a

tensor([[ 0.6855,  1.4085, -0.8217],
        [ 0.5253, -1.5035,  1.0939]])

In [5]:
%timeit t.a

70 ns ± 0.0726 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f92f82dfeb0>
├── 'a' --> tensor([[-2.4136, -0.5851,  0.1852],
│                   [-0.0716,  1.1884, -0.3052]])
└── 'x' --> <FastTreeValue 0x7f92f82dfdc0>
    └── 'c' --> tensor([[-1.8362, -2.1066, -0.7294, -1.0983],
                        [ 0.6755,  0.3028, -0.6359,  0.0524],
                        [-1.0733,  0.6447,  0.1886, -2.0146]])

In [7]:
%timeit t.a = new_value

72.8 ns ± 0.0663 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939]]),
    x: Batch(
           c: tensor([[-1.8362, -2.1066, -0.7294, -1.0983],
                      [ 0.6755,  0.3028, -0.6359,  0.0524],
                      [-1.0733,  0.6447,  0.1886, -2.0146]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6855,  1.4085, -0.8217],
        [ 0.5253, -1.5035,  1.0939]])

In [11]:
%timeit b.a

56.9 ns ± 0.0141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.0578, -0.0511,  0.5297],
               [ 1.2325,  0.0517, -1.1230]]),
    x: Batch(
           c: tensor([[-1.8362, -2.1066, -0.7294, -1.0983],
                      [ 0.6755,  0.3028, -0.6359,  0.0524],
                      [-1.0733,  0.6447,  0.1886, -2.0146]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.0665 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

831 ns ± 0.341 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 9.16 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 463 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 368 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f924dfc5d60>
├── 'a' --> tensor([[[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]],
│           
│                   [[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]],
│           
│                   [[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]],
│           
│                   [[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]],
│           
│                   [[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]],
│           
│                   [[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]],
│           
│                   [[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]],
│           
│                   [[ 0.6855,  1.4085, -0.8217],
│                    [ 0.5253, -1.5035,  1.0939]]])
└── 'x' --> <FastTreeValue 0x7f92f82d8430>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 42.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9254bea070>
├── 'a' --> tensor([[ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939],
│                   [ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939],
│                   [ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939],
│                   [ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939],
│                   [ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939],
│                   [ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939],
│                   [ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939],
│                   [ 0.6855,  1.4085, -0.8217],
│                   [ 0.5253, -1.5035,  1.0939]])
└── 'x' --> <FastTreeValue 0x7f9254bdf0d0>
    └── 'c' --> tensor([[-1.8362, -2.1066, -0.7294, -1.0983],
                        [ 0.6755,  0.3028, -0.6359,  0.0524],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 44.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 79.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]],
       
               [[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]],
       
               [[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]],
       
               [[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]],
       
               [[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]],
       
               [[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]],
       
               [[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]],
       
               [[ 0.6855,  1.4085, -0.8217],
                [ 0.5253, -1.5035,  1.0939]]]),
    x: Batch(
           c: tensor([[[-1.8362, -2.1066, -0.7294, -1.0983],
                       [ 0.6755,  0.3028, -0.6359,  0.0524],
                       [-1.0733,  0.6447,  0.1886, -2.0146]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 155 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939],
               [ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939],
               [ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939],
               [ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939],
               [ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939],
               [ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939],
               [ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939],
               [ 0.6855,  1.4085, -0.8217],
               [ 0.5253, -1.5035,  1.0939]]),
    x: Batch(
           c: tensor([[-1.8362, -2.1066, -0.7294, -1.0983],
                      [ 0.6755,  0.3028, -0.6359,  0.0524],
                      [-1.0733,  0.6447,  0.1886, -2.0146],
                      [-1.8362, -2.1066, -0.7294, -1.0983],
                      [ 0.6755,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 299 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

313 µs ± 3.91 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
